# MMVC - Mineral Miner VC

Use AI (large language models) to evaluate a mining investment pitch.

This work is meant to show how AI can greatly accelerate analysis for investment opportunities that professionals take years to master. I believe there's a window of opportunity for tech-savvy novices to use AI to level the investment playing field. Get scrappy!

NOTE: I am not an investment professional, and this is not investment advice. I am only sharing my opinion about what I do.

Steps:
1. Get pitch documents loaded.
2. Vectorize data.
3. Use AI to generate a list of questions.
4. Perform `RAG` analysis on questions.
5. Review Results (human feedback is CRITICAL!)

**About me**: Hi! I'm David. I'm an AI engineer following the resources (commodities) industries.

I'm busy automating manual work and improving our understanding of mining ventures so investors don't get ruged in today's critical mineral super-cycle.

I'm on [linkedin](https://linkedin.com/in/davidimprovz)  |  [twitter](https://twitter.com/d_comfe)  |  [email](mailto:drifft.hello@gmail.com)  |  [my free newsletter](https://drifft.beehiiv.com)

## 1. Pre-reqs

In [ ]:
# if you're on google colab, you can run this cell
%pip install -qU \
    langchain \
    openai \
    pypdf \
    cohere \
    tiktoken \
    llama-index \
    sentence-transformers \
    pandas

In [2]:
from getpass import getpass
import warnings
import os
from time import time
from pprint import pprint
warnings.filterwarnings('ignore')
from pathlib import Path
import pandas as pd


For simplicity, we'll use OpenAI. But if you were doing this for confidential investment opportunities, you would want your own AI setup so your data doesn't leak.  

If your personal hardware is fast enough, you could set up and run [https://ollama.ai/](https://ollama.ai/) or [https://lmstudio.ai/](https://lmstudio.ai/) with opensource models like Llama2 or Mistral.


In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

os.environ['OPENAI_API_KEY'] = getpass() # enter your openai API key

openai_llm = ChatOpenAI(
    model="gpt-3.5-turbo", # try gpt-4 if you want the best results
    temperature=0.1, # control how creative the ai can be
)

··········


## 2. Read Pitch and Vectorize It
We need to get our pitch deck data and convert it into something called vectors. Vectors are number representations of the text that help us find all data relevant to any question we ask.

In [4]:
# uncomment and run if you're using Google Colab to store results to your Drive

from google.colab import files
from google.colab import drive

drive_dir = '/content/drive/'
drive.mount(drive_dir)

from pathlib import Path
file_dir = drive_dir + 'My Drive/[...]' # [your destination]

if Path(file_dir).exists():
    %cd $file_dir
    !ls
else:
    print('No folder found')

Mounted at /content/drive/
/content/drive/My Drive/ColabNotebooks/Projects/Drifft
data  langchain_llama_mineral_pitches.ipynb  mineral_invest_ai_analyst.ipynb  sentence_index


### Using Llama-Index
The most popular vector store framework that provides several ways to extract infromation from a vector database.

In [5]:
from llama_index import ServiceContext
from llama_index import VectorStoreIndex
from llama_index.llms import OpenAI
from llama_index import SimpleDirectoryReader

# first we have to load our documents
llama_idx_docs = SimpleDirectoryReader(input_dir="./data").load_data()

print(len(llama_idx_docs), "\n") # get # of pages
print(llama_idx_docs[0]) # show the first doc

#### Basic Index - quick and easy solution, but not the best

In [ ]:
# let's now create a vector store for the data that
# our AI will use to extract information relevant to
# our questions. Depending on the size of your pitch
# deck and your hardware (e.g., standard laoptop)
# this can take a few min.

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

service_context = ServiceContext.from_defaults(
     llm=openai_llm, # from langchain
     embed_model="local:BAAI/bge-small-en-v1.5"
)

index = VectorStoreIndex.from_documents(
    llama_idx_docs,
    service_context=service_context
    )


In [ ]:
# let's test to see if our index query works
question = "What is the name of the gold mining company?"
query_engine = index.as_query_engine()
response = query_engine.query(question)

pprint(str(response))

The length of the investment payback period is not mentioned in the given context information.


#### Sentence Window Index
Retrieves more information than the basic query by loading documents before and after the one that the similiarity search finds pertaining to your quesiton.

In [7]:
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.storage import StorageContext
from llama_index.indices.loading import load_index_from_storage

def build_sentence_window_index(
        documents,
        llm,
        embed_model="local:BAAI/bge-small-en-v1.5",
        save_dir="./sentence_index"
    ):
    # create the sentence window node parser w/ default settings
    print('building node parser')
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=3,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
        )

    print('building service context')
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
        )

    if not os.path.exists(save_dir): # save the index to your directory to avoid making extra API calls (i.e, pay more) to openai
        print('building vector store')
        sentence_index = VectorStoreIndex.from_documents(
            documents,
            service_context=sentence_context
            )

        print('saving vector store')
        sentence_index.storage_context.persist(
            persist_dir=save_dir
        )

    else:
        print('loading vectore store')
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
            )

    return sentence_index


In [ ]:
# this cell can take a while depending on hardware
# and size of the pitch deck. It will build the
# vector database from your pitch document.

sentence_index = build_sentence_window_index(
    llama_idx_docs,
    openai_llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="./sentence_index"
)


In [9]:
# now we build a query engine for the vector database just created

from llama_index import VectorStoreIndex, get_response_synthesizer
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import VectorIndexRetriever
from llama_index.indices.postprocessor import SimilarityPostprocessor
from llama_index.postprocessor import MetadataReplacementPostProcessor
from llama_index.postprocessor import SentenceTransformerRerank


def get_sentence_window_query_engine(
        sentence_index,
        type='query', # or retrieve
        similarity_top_k=10, # play with this value to tweak how much data gets passed to the ai
        rerank_top_n=2
    ):

    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n,
        model="BAAI/bge-reranker-base"
        )

    if type == 'query': # for when we want to do queries and have AI synthesize a result
        sentence_window_engine = sentence_index.as_query_engine( # as_retriever( if you only want to do document retrieval
            similarity_top_k=similarity_top_k,
            node_postprocessors=[postproc, rerank],
            )

    elif type == 'retrieve': # for when we only want to get the documents back
        retriever = VectorIndexRetriever(
            index=sentence_index,
            similarity_top_k=10,

        )
        # assemble query engine with no_text response mode
        sentence_window_engine = RetrieverQueryEngine(
            retriever=retriever,
            response_synthesizer=get_response_synthesizer(),
            node_postprocessors=[postproc, rerank],
        )

    else:
        raise ValueError(f'unknown retriever engine type requested: {type}')

    return sentence_window_engine


In [15]:
# finally..we can test it out with a question

question = """What is the name of the gold mining company?"""

sentence_window_engine = get_sentence_window_query_engine(sentence_index)
window_response = sentence_window_engine.query(question)

print(str(window_response))

The name of the gold mining company is Kinross Gold Corporation.


#### Automerge Index
Another variation on context windows, which will grab slightly different information for us to compare results with the previous version to see if we get more relevant info from our questions.

In [ ]:
from llama_index.node_parser import HierarchicalNodeParser
from llama_index.node_parser import get_leaf_nodes

def build_automerging_index(
        documents,
        llm,
        embed_model="local:BAAI/bge-small-en-v1.5",
        save_dir="merging_index",
        chunk_sizes=None,
    ):

    chunk_sizes = chunk_sizes or [2048, 512, 128]

    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)

    nodes = node_parser.get_nodes_from_documents(documents)

    leaf_nodes = get_leaf_nodes(nodes)

    merging_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        )

    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes,
            storage_context=storage_context,
            service_context=merging_context
            )

        automerging_index.storage_context.persist(persist_dir=save_dir)

    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=merging_context,
            )

    return automerging_index


automerging_index = build_automerging_index(
    llama_idx_docs,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [ ]:
from llama_index.retrievers import AutoMergingRetriever
from llama_index.query_engine import RetrieverQueryEngine

def get_automerging_query_engine(
        automerging_index,
        similarity_top_k=12,
        rerank_top_n=2,
    ):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)

    retriever = AutoMergingRetriever(
        base_retriever,
        automerging_index.storage_context,
        verbose=True
        )

    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n,
        model="BAAI/bge-reranker-base"
        )

    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever,
        node_postprocessors=[rerank]
        )

    return auto_merging_engine

In [ ]:
question = """
What is the name of the gold mining company,
how much are they requesting,
and what is this investment for?
"""

automerging_query_engine = get_automerging_query_engine(automerging_index)
auto_merging_response = automerging_query_engine.query(question)

pprint(str(auto_merging_response))

In [ ]:
# quickly compare how each query engine performs
# on your docs and questions.

test_query = "Where is the mine located, and how accessible is it?"
sentence_window_answer = sentence_window_engine.query(test_query)
auto_merge_answer = automerging_query_engine.query(test_query)

# it seemed an eyeball inspection on the test doc
# showed the sentence window engine performs better
# (more relevant).
pprint(sentence_window_answer), pprint(auto_merge_answer)

## 3. Generate Questions
Use AI to generate a set of questions that make sense for us to ask about this potential investment.

In [13]:
# let's have AI generate some questions that a professional mining investor would ask
from langchain.prompts import PromptTemplate

get_questions_template = PromptTemplate.from_template(
    """
    You are an expert investor specializing in mineral mining investments.
    Your job is to create a comprehensive set of questions
    to be used for analyzing a potential investment. Use the context
    below to generate a list of questions that will allow
    the investor to make an informed yes/no decision as well as a
    counter-offer to the business seeking the investment.

    %CONTEXT
    {context}
    """
    )

question_generator = get_questions_template | openai_llm | StrOutputParser()
# with some work, you could use the query engine on the pitch deck to get more relevant context as well
generated_questions = question_generator.invoke({"context":"This investment is for a gold mining project in Nome, Alaska."})
generated_questions = generated_questions.split('\n')

for question in generated_questions:
    pprint(question)

('1. What is the estimated size and quality of the gold deposit in Nome, '
 'Alaska?\n'
 '2. Has there been any previous exploration or mining activity in the area? '
 'If so, what were the results?\n'
 '3. What is the current market price of gold and how has it been trending in '
 'recent years?\n'
 '4. What is the projected lifespan of the gold mine in Nome, Alaska?\n'
 '5. What are the estimated production costs per ounce of gold?\n'
 '6. Are there any environmental regulations or restrictions in place that '
 'could impact the mining operations?\n'
 '7. What is the infrastructure like in Nome, Alaska? Are there sufficient '
 'transportation and logistics capabilities to support the mining operations?\n'
 '8. What is the political and social stability in Nome, Alaska? Are there any '
 'potential risks or challenges related to local communities or government?\n'
 '9. What is the experience and track record of the management team involved '
 'in the gold mining project?\n'
 '10. What 

In [ ]:
# if you want to use some pre-fab questions to evaluate the performance, be my guest
prefab_questions = [
    "What is the location and ownership structure of the gold mine in Alaska?",
    "Where is the mine located, and how accessible is it?",
    "What is the current production rate and expected future growth potential of the mine?",
    "What is the quality and grade of the ore being produced by the mine?",
    "How long has the mine been operating, and what is its track record of profitability",
    "What are the current and projected expenses associated with running the mine, including costs for labor, equipment, and maintenance?",
    "Are there any known risks or challenges facing the gold mine in Alaska that could impact its future performance?",
    "What is the current market demand for gold, and how does this demand impact the potential profitability of the mine?",
    "What is the competitive landscape in the gold mining industry in Alaska, and how does this compare to other regions?",
    "What is the management team's experience and track record in the gold mining industry?",
    "Are there any significant projects or initiatives planned for the future development of the mine?",
    "What are the transportation costs and logistics like for equipment, supplies, and personnel?",
    "Are there any nearby infrastructure or communities that could provide support services?",
    "What is the geological history of the area, and how does it affect the mineralization of the deposit?",
    "What is the grade and quality of the gold deposit, and how does it compare to other similar deposits in the region?",
    "Are there any other minerals or commodities present in the deposit that could provide additional value?",
    "What stage of exploration and development is the mine at, and what are the planned next steps?",
    "Have there been any previous exploration or mining activities at the site, and if so, what were the results?",
    "Are there any known geological or environmental challenges that could affect the mine's operation or profitability?",
    "What is the proposed mining plan and schedule, including the rate of production and the life of mine?",
    "What are the estimated operating costs, including labor, equipment, energy, and supplies?",
    "Are there any potential bottlenecks or constraints in the mining process that could affect profitability?",
    "What is the proposed processing and refining plan for the gold ore, including the type of equipment and methods used?",
    "What are the estimated processing and refining costs, and how do they compare to similar operations in the region?",
    "Are there any potential issues with the processing or refining stages that could affect profitability?",
    "Are there any existing infrastructure or support services available at the mine site, such as power, water, and transportation?",
    "What are the estimated costs for building any necessary infrastructure or support services?",
    "How will the mine site be maintained and monitored to ensure safe and efficient operation?",
    "What is the potential environmental impact of the mine, including the effects on water, air, and land quality?",
    "How will the mine owner mitigate any negative environmental or social impacts, and what are the estimated costs?",
    "Are there any potential risks to the local community or indigenous peoples that could affect the project's success or profitability?",
    "What are the relevant legal and regulatory requirements for mining in the jurisdiction, and how does the mine comply with them?",
    "Are there any potential legal or regulatory risks that could affect the project's success or profitability?",
    "How will the mine owner ensure compliance with all relevant laws and regulations throughout the life of the mine?",
    "What are the estimated financial projections for the mine, including revenue, operating costs, capital expenditures, and cash flow?",
    "How do the financial projections account for potential risks and uncertainties, such as changes in gold prices or geological conditions?",
    "Are there any potential financial risks or challenges that could affect the project's success or profitability?",
    "What is the planned exit strategy for the mine owner, including potential sale of the asset or continued operation?",
    "How will the exit strategy be affected by factors such as changes in gold prices or geological conditions?",
    "Are there any potential risks or challenges that could affect the exit strategy and profitability?"
]

## 4. Reason over the questions
Use the query engine to return infromation that's similar and then ask the AI model to perform some analysis on it.

In [ ]:
# let's set up our document retriever

sentence_window_doc_retriever = get_sentence_window_query_engine(sentence_index, 'retrieve')

def retrieve_docs(query):
    response = sentence_window_doc_retriever.query(query)
    text = [node.text for node in response.source_nodes]
    return '\n'.join(text)

In [11]:
# define a prompt for the question/answer chain

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

investment_question_prompt = PromptTemplate.from_template(
    """
    You are an expert investor specializing in mineral investments.
    Your job is to provide succinct answers to the question below
    given the context.

    %CONTEXT {context}

    %QUESTION {question}
    """
)

setup_and_retrieval = RunnableParallel(
    {"context": retrieve_docs, "question": RunnablePassthrough()}
)

investment_qa_chain = setup_and_retrieval | investment_question_prompt | openai_llm | StrOutputParser()


In [18]:
# let's ask the ai to take each question and answer them
# one by one.

# depending on how long your list is, you could incur
# charges. With ~40 questions and gpt-3.5-turbo, I paid
# about $.07.

print(f'the AI will be asked to answer {len(generated_questions)} questions.')
answers = investment_qa_chain.batch(generated_questions)

the AI will be asked to answer 20 questions.


In [ ]:
# let's review each answer next to its question
responses = pd.DataFrame({'question':generated_questions, 'answer':answers,})

# save to disk so we don't have to run again.
responses.to_csv('./data/ai_investor_responses.csv', index=False)

for _, qa_s in responses.iterrows():
    pprint(qa_s.loc['question'].strip())
    pprint(qa_s.loc['answer'].strip())
    print('\n')

In [38]:
# now we'll combine the ?s and answers
# and ask the AI whatever questions we
# want to know.

summary_template = PromptTemplate.from_template("""
    You are an expert venture capitalist specializing in the mineral mining industry.
    Review the following question and answer summaries and then carefully reason over
    them to answer the question.

    %SUMMARIES
    {summaries}

    %QUESTION
    {question}
    """)

helper_chain = summary_template | openai_llm | StrOutputParser()

In [39]:
# first we'll create the summaries from the responses we got.

summaries = ' '.join(['Question: ' + question + ' Answer: ' + answer for  idx, question, answer in responses.itertuples()])
pprint(summaries[:500])

('Question: 1. What is the estimated size and quality of the gold deposit in '
 'Nome, Alaska? Answer: The estimated size of the gold deposit in Nome, Alaska '
 'is not explicitly mentioned in the given context. However, the indicated '
 'resource estimate is 79,000 ounces and the inferred resource estimate is '
 '155,000 ounces. Question: 2. Has there been any previous exploration or '
 'mining activity in the area? If so, what were the results? Answer: Yes, '
 'there have been previous exploration and mining activities')


In [40]:
# the length of the summeries is quite large
# let's make sure the model can run the number
# of words we have

import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-0613")
num_tokens = len(encoding.encode(summaries))
num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>

print(f"using {num_tokens} tokens -> remaining tokens: {4096 - num_tokens}")

using 1249 tokens -> remaining tokens: 2847


In [41]:
# imagine you're an investor. let's ask ai to produce a
# series of questions for a followup meeting, which
# aren't answered in the pitch.

# if you're using OpenAI, you can check how much
# the chain costs to run

from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:

    question = """what are the top 3-4 most important questions
                to ask the entrepreneur about this pitch which are not
                addressed in the summaries?"""

    answer = helper_chain.invoke({'summaries':summaries, 'question':question})

    print(cb, '\n\n')

pprint(answer)

# notice that we are reaching the word (i.e., token) limit of this AI model, so we
# might want to use one that allows for a lot more. Opensource models give us a lot
# of options here and I leave it for you to explore.

Tokens Used: 1414
	Prompt Tokens: 1332
	Completion Tokens: 82
Successful Requests: 1
Total Cost (USD): $0.0021620000000000003 


('1. What is the estimated cost of establishing and operating the gold mine in '
 'Nome, Alaska?\n'
 '2. What is the current market price of gold and how has it been trending in '
 'recent years?\n'
 '3. What is the projected lifespan of the gold mine in Nome, Alaska?\n'
 '4. Are there any potential risks or challenges specific to gold mining in '
 'Nome, Alaska, such as extreme weather conditions or geological complexities?')


In [43]:
# evaluate the project on its own merits
with get_openai_callback() as cb:

    question = """There is never an idea investment decision where you have
    all the data. Knowing that, how would you evaluate the project? You MUST
    make a decision.

    Based on the limited data, would it make a good investment based on what
    you see in the summaries?

    Answer as a yes / no, and provide some context for how you would make
    the decision under uncertainty."""

    answer = helper_chain.invoke({'summaries':summaries, 'question':question})

    print(cb, '\n')

pprint(answer)


Tokens Used: 1709
	Prompt Tokens: 1386
	Completion Tokens: 323
Successful Requests: 1
Total Cost (USD): $0.0027249999999999996 

('Based on the limited data provided in the summaries, it is difficult to make '
 'a definitive evaluation of the investment opportunity in the gold mining '
 'project in Nome, Alaska. However, based on the available information, I '
 'would lean towards a tentative "yes" for considering it as a potential '
 'investment.\n'
 '\n'
 'The presence of previous exploration and mining activities in the area, '
 'resulting in over 120,000 ounces of gold, suggests the presence of a viable '
 'gold deposit. Additionally, the indicated and inferred resource estimates of '
 '79,000 and 155,000 ounces respectively indicate a potentially significant '
 'gold deposit.\n'
 '\n'
 'However, there are several key pieces of information that are missing, such '
 'as the estimated size and quality of the gold deposit, the current market '
 'price of gold and its trend, the projec

In [44]:
# tell the investor how to improve the pitch
with get_openai_callback() as cb:

    question = """Based on the summaries, how could the investor improve
    their pitch deck for this investment opportunity?"""

    answer = helper_chain.invoke({'summaries':summaries, 'question':question})

    print(cb, '\n\n')

pprint(answer)


Tokens Used: 2174
	Prompt Tokens: 1322
	Completion Tokens: 852
Successful Requests: 1
Total Cost (USD): $0.003687 


('Based on the summaries, the investor could improve their pitch deck for this '
 'investment opportunity by including the following information:\n'
 '\n'
 '1. Estimated size and quality of the gold deposit in Nome, Alaska: The '
 'investor should provide specific information about the estimated size and '
 'quality of the gold deposit in order to assess the potential profitability '
 'of the mining project.\n'
 '\n'
 '2. Previous exploration and mining activity in the area: The investor should '
 'provide details about the results of previous exploration and mining '
 'activities in the area to understand the historical success and potential '
 'risks associated with the project.\n'
 '\n'
 '3. Current market price of gold and its trend: The investor should include '
 'information about the current market price of gold and how it has been '
 'trending in recent years to 

In [45]:
# ask AI to help me create a counteroffer or figure out a way to get rights to the land and resources
with get_openai_callback() as cb:

    question = """Based on the lack of information provided in this pitch deck,
                and knowing that they're offering a 10% stake for $5M,
                can you please produce a range of counteroffers for low-ball,
                reasonable, and high-water offers?

                If you don't have enough information, just throw a number out
                there."""

    answer = helper_chain.invoke({'summaries':summaries, 'question':question})

    print(cb, '\n\n')

pprint(answer)


Tokens Used: 1591
	Prompt Tokens: 1373
	Completion Tokens: 218
Successful Requests: 1
Total Cost (USD): $0.0024955000000000003 


('Based on the lack of information provided in the pitch deck, it is difficult '
 'to accurately determine counteroffers for the gold mining project in Nome, '
 'Alaska. However, based on the given context that they are offering a 10% '
 'stake for $5M, we can provide a range of counteroffers as follows:\n'
 '\n'
 'Low-ball offer: $2M - $3M for a 10% stake. This offer reflects a more '
 'cautious approach due to the limited information provided.\n'
 '\n'
 'Reasonable offer: $4M - $6M for a 10% stake. This offer takes into account '
 'the potential risks and uncertainties associated with the project, as well '
 'as the lack of detailed information.\n'
 '\n'
 'High-water offer: $7M - $9M for a 10% stake. This offer assumes a more '
 'optimistic outlook for the project and considers the potential upside and '
 'future profitability of the gold mining operations

## 5. Results

1. Whether you're an investor or entrepreneur, AI can reason over the information and help you find and fill in holes.

2. To get more out of the AI, you have to provide great questions. So being skilled at asking good questions will be a fantastic skill in the future with even more powerful AI.

3. Recgonize that you will get *hallucinations* from the AI, and sometimes the query engine won't find all the data in your file. This goes back to point #2. The better you can write questions, and the more accurate your query engine assistant is, the more accurate your responses will get. This is a field of active research and development.

4. Always verify the data with human feedback.

You're thinking: "I could do a lot more with this concept." And you're 100% right! Imagine reasoning over an entire mining portfolio!

Contact me if you're interested in exploring ideas: [drifft.hello@gmail.com](mailto:drifft.hello@gmail.com)

Also, there's a reason VC's are chasing new tech in the resources industries. I write about it all in my [weekly newsletter](https://drifft.beehiiv.com) - free.

Also on [linkedin](https://linkedin.com/in/davidimprovz)  |  [twitter](https://twitter.com/d_comfe)
